# Main Header

## TODOs

- create a desc for each function
- comment code properly
- append code with full model -> "File, Download as Python"
- Computationally intensive! / Use Spark on AWS
    - https://eu-central-1.console.aws.amazon.com/ec2/v2/home?region=eu-central-1#LaunchInstanceWizard:
    - http://blog.insightdatalabs.com/spark-cluster-step-by-step/
    - http://blog.insightdatalabs.com/jupyter-on-apache-spark-step-by-step/
    - http://stackoverflow.com/questions/37327021/using-jupyter-notebook-on-spark-on-emr

## Getting Started

### Purpose

- Get started with neural nets, Convolutions?, Fully-connected layers, activations . not eventually but anyway
- See a thing learn is exciting (05 smartcab)
- The field of ML I know least
- NOT: 2d inputs, convolutions / different input preprocessing // too: I.astype(np.float).ravel()
- Get aquainted with AWS EC2 Spark Cluster

### Credits and Thanks

- Neural Networks
    - Andrew Trask https://iamtrask.github.io
    - Sebastian Raschka http://sebastianraschka.com/Articles/2015_singlelayer_neurons.html
- Keras
    - Francois Chollet https://keras.io https://github.com/fchollet/keras/tree/master/examples 
- Deep Q Learning
    - Deep Mind: Playing Atari with Deep Reinforcement Learning https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf
    - Tambet Matiisen https://www.nervanasys.com/demystifying-deep-reinforcement-learning/ https://github.com/tambetm/simple_dqn/blob/master/src/replay_memory.py
    - Eder Santana http://edersantana.github.io/articles/keras_rl/
    - Ben Lau https://yanpanlau.github.io/2016/07/10/FlappyBird-Keras.html
- Policy Gradients
    - Andrej Karpathy http://karpathy.github.io/2016/05/31/rl/
    - Denny Britz https://github.com/dennybritz/reinforcement-learning/tree/master/PolicyGradient/
    - Open AI Gym https://gym.openai.com/docs/rl#policy-gradients
    - David Silver https://www.youtube.com/watch?v=KHZVXao4qXs

### Main Dependencies

We are working with Open AI Gym (https://gym.openai.com/) as a training environment for our to-be-defined AI agent.

Lunar Lander environment (https://gym.openai.com/envs/LunarLander-v2) is particularily appealing to me. It is based on box2d, which simulates real life physics. Charming! And, with its 1D input state vector, it is a first step for creating and tuning an AI agent, before proceeding with convolution preprocessing of 2D inputs.

The environment home page says the following:

*"Landing pad is always at coordinates (0,0). Coordinates are the first two numbers in state vector. Reward for moving from the top of the screen to landing pad and zero speed is about 100..140 points. If lander moves away from landing pad it loses reward back. Episode finishes if the lander crashes or comes to rest, receiving additional -100 or +100 points. Each leg ground contact is +10. Firing main engine is -0.3 points each frame. Solved is 200 points. Landing outside landing pad is possible. Fuel is infinite, so an agent can learn to fly and then land on its first attempt. Four discrete actions available: do nothing, fire left orientation engine, fire main engine, fire right orientation engine."*

Model building and training is done with Keras (https://keras.io). This modular, minimalist library makes ANN life as easy as it can get, and in plus runs on both Theano and Tensorflow backends.

## Chapter 1: The Starting Point: Q-Learning, or: Evaluating Bellman Equations from data

Q learning is about revisiting states. We are in a specific state s at time t, and because the state space is sufficiently small, we might discover that the agent has already been in s before. It therefore has made an experience for s by taking an action, and collecting a reward or punishment. All this is stored in the agent's "memory" (possible a dictionary of dictionaries, main keys being the states, values being the actions (keys) and their values). We want the agent to take advantage of this "memory": We look up the expected lifetime rewards per each possible action in s (a.k.a. action-value function, q values), select the maximum q value, and execute the chosen action. 

We now have fresh evidence about the consequence of a specific action in a specific state: We know the initial state *s_t*, we know the selected action *a_t*, we know the reward *r_t*, and we know the new state this all lead to, s_t1. This knowledge we now use to update the agent's memory: We calculate a new q value for *s_t* by taking the observed reward *r_t*, and adding to it the discounted maximum q value for *s_t1*. The difference to the old q value is the new q value for action *a_t* in state *s_t*.

### Why Q-Learning Often Does Not Work: Exploding State Spaces

Revisiting states is often not possible, even in the long run, because there are simply too many combinations of relevant inputs which constitute a state. Just think a small number of inputs, each input being a floating number with 4 digits. Even this small setting is creating a large amount of combinations: the state space explodes. Revisiting states is very unlikely, we will need a huge number of trials to generate memory updates. As a consequence, learning is slow, or even not happening.

In order to illustrate the point, we are going to set up a basic q-learning algorithm:

#### Namespace

In [1]:
import numpy as np
import gym

#### Global Seed

In [2]:
np.random.seed(3293734)

#### Set Hyperparameters

- *GAMMA* is the factor by which future expected rewards are discounted
- *ALPHA* is the learning rate TODO
- *N_EPISODES* denotes the maximum number of episodes an epoch will embrace
- *Q_TABLE* is the agent's memory. The states are the keys, and dictionaries of actions and their respective values are the values
- *VALUE_INIT* is the initial value for the actions of a state, once the state is visited the first time. It is set to zero.

In [3]:
GAMMA = 0.99
ALPHA = 0.1
N_EPISODES = 10
Q_TABLE = {}
VALUE_INIT = 0

#### Prepare Environment

We create an instance (*ENV*) of the Lunar lander environment. Its input dimensions *INPUT_DIM* are obtained by resetting the environment, and the actions by the environment method *action_space*.

In [4]:
ENV = gym.make("LunarLander-v2")
INPUT_DIM = ENV.reset().shape[0]
N_ACTIONS = ENV.action_space.n
ACTIONS = np.arange(0, N_ACTIONS)

[2016-10-24 22:02:09,325] Making new env: LunarLander-v2


#### Build Training Epoch

We are going to create the main building block of this exercise: The training block for one epoch.

In [5]:
def train_ql(render):
    # Start a new epoch
    # Bookkeeping
    # Init counter for how many times we revisited states
    revisiting_states = 0 
    # Count # of episodes, # successes (episode with >= 100 reward), and if the environment is solved
    episode = 0; success = 0; solved = False 

    # Play through episodes
    for episode in range(N_EPISODES): 
        # Reset environment and get first observation
        x_t = ENV.reset() 
        # Take only the current observation as state, in order to keep the state space as small as possible
        s_t = tuple(x_t) 
        done = False

        # Start an episode
        while not done:             
            # Reset the environment for a new gaming step
            if render: ENV.render() 
            
            # Look up the action with the highest value at s_t, as well as its value
            a_t, q, r_s  = best_action(s_t)

            # Observe after action a_t
            x_t, r_t, done, info = ENV.step(a_t) 
            # Again, take only the current observation
            s_t1 = tuple(x_t) 
            
            # Look up the action with the highest value at s_t1, as well as its value
            a_t1, Q_sa, _ = best_action(s_t1) 
            
            # Look up the action with the highest value at s_t1, as well as its value
            Q_TABLE[s_t][a_t] = q + ALPHA * (r_t + GAMMA * Q_sa - q) 
            
            # Update
            s_t = s_t1
            revisiting_states += r_s
            
            # Bookkeeping
            if r_t >= 100: success += 1
            if r_t >= 200: solved = True
                
    print "Number of states in Q table: {}, Number of revisited states: {}, Successes {}, Solved {}".format(len(Q_TABLE), revisiting_states, success, solved)

# Create helper function to initialize and query Q-table
def best_action(state):
    # Create init scenario for table queries at t and t1
    if state not in Q_TABLE or sum(Q_TABLE[state].values()) == 0: 
        # Bookkeeping: Init revisiting states counter
        revisit_state = 0 
        # Init q function
        q_function = {} 
        for A in ACTIONS: q_function[A] = VALUE_INIT
        Q_TABLE[state] = q_function 
        # Do random action
        action = np.random.choice(ACTIONS, 1)[0] 
    else: 
        revisit_state = 1
        # Select action according to max q
        action = max(Q_TABLE[state], key=Q_TABLE[state].get) 
    # Get q value for action selected
    q = Q_TABLE[state][action] 
    return action, q, revisit_state

#### Train Model

In [6]:
train_ql(False)

Number of states in Q table: 882, Number of revisited states: 0, Successes 0, Solved False


## Chapter 2: What to Do? Do Not Update the Q Function, But the Q  Function Estimator: Deep Q Learning

In this situation, we replace the "revisiting states" by a function approximator: 

We let a Artifical Neural Net (ANN) estimate the q function for the state s the agent is visiting at time t. 

Once we performed the action based on the maximum of the q function (just the action with the highest expected lifetime reward at time t), we know the reward, and the subsequent state. 

Based on this, we are able to update the agent's memory. But this time, we do not update the q function directly. Instead, we are updating the ANN, which means that we are updating the weights used in the ANN. And that is how exactly:

- At time t, we already know the estimation of the q function for state s_t: We used it to pick an action a_t accordingly. Read again: this is the **estimation** of the q function.

- After action a_t, we know *s_t*, *a_t*, *r_t* and *s_t1*. This allows us to update the q function, **but only for the action taken**. We take r_t, and add to it the discounted expected lifetime reward, in other words we let the ANN estimate the q function for state s_t1. 

- For the action taken, we can update the q value now. All the other actions are not performed, we do not know about the reward, or a subsequent state *s_t1*. So, we cannot learn for those. This updated q function is the **target**.

- We feed the error, which is the difference between the estimation and the target.

- We backpropagate the error through the network, such that the weights are updated.

Next time we estimate the q function for another state s, we have updated weights

### Understanding the Gist of ANNs

Firstly, I wanted to understand, what a neural network really does. [iamtrask]'s excellent toy examples helped me understand it completely. I create a network with one hidden layer, and the output layer, both sigmoid activated, which returns probablities for each of the outputs.

layer_2_wloss ("weighted loss") is where the magic happens: Each output loss is multiplied by the slope / gradient of the predicted value on the sigmoid curve.

#### Create A Basic ANN With Only Numpy

In [30]:
# Create the sigmoid function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Create the sigmoid function's derivative
def sigmoid_derivative(x):
    return x * (1 - x)

hidden_dims = np.arange(1, 4)
training_steps = 100

alphas = [0.01, 0.1, 1, 10, 100, 1000, 1000]

X = np.random.randint(2, size=(4, 3))
y = np.random.randint(2, size=(4, 1))

# Test the hidden dimensions
for hidden_dim in hidden_dims: 
    
    # Initialize 1st set of weights
    W1 = np.random.rand(X.shape[1], hidden_dim) 

    # Initialize 2nd set of weights
    W2 = np.random.rand(hidden_dim, y.shape[1]) 
    
    # Test the alphas
    for alpha in alphas: 
        
        for i in range(training_steps):
            # Forward propagate
            # Initialize hidden layer (fully connected)
            layer_1 = np.dot(X, W1)
            # Apply sigmoid activation
            layer_1 = sigmoid(layer_1) 
            
            # Initialize output layer(fully connected)
            layer_2 = np.dot(layer_1, W2) 
            # Apply sigmoid activation 
            layer_2 = sigmoid(layer_2) 

            # Get loss
            layer_2_loss = y - layer_2 

            ''' Apply SGD to the loss: the more certain the estimate, the less weighted it will get: 
                The gradient at the extremes is smaller than in the middle
            '''
            layer_2_wloss = layer_2_loss * sigmoid_derivative(layer_2) # element-wise multiplication!
            
            # Backpropagate
            # Compute the effect of the hidden layer to the weighted loss
            layer_1_loss = np.dot(layer_2_wloss, W2.T) 

            # Apply SGD
            layer_1_wloss = layer_1_loss * sigmoid_derivative(layer_1) 
                
            # Update weights
            W2 += alpha * np.dot(layer_1.T, layer_2_wloss)
            W1 += alpha * np.dot(X.T, layer_1_wloss)

            if i == 1: first_error = np.mean(np.abs(layer_2_loss))
            if i == training_steps - 1: print "Hidden Size {}, alpha {}: Final avg loss {}, Improvement {}".format(
                                                '%2s' % hidden_dim, \
                                                '%4s' % alpha, \
                                                '%16s' % np.mean(np.abs(layer_2_loss)), \
                                                np.mean(np.abs(layer_2_loss)) - first_error)

Hidden Size  1, alpha 0.01: Final avg loss   0.419350997248, Improvement -0.00832343220248
Hidden Size  1, alpha  0.1: Final avg loss   0.368040757929, Improvement -0.0504266722845
Hidden Size  1, alpha    1: Final avg loss   0.210627762563, Improvement -0.153751679047
Hidden Size  1, alpha   10: Final avg loss   0.140561208153, Improvement -0.0633980645893
Hidden Size  1, alpha  100: Final avg loss   0.128919879958, Improvement -0.0106878878033
Hidden Size  1, alpha 1000: Final avg loss   0.126105107208, Improvement -0.00260340476934
Hidden Size  1, alpha 1000: Final avg loss   0.125783531658, Improvement -0.000311005371323
Hidden Size  2, alpha 0.01: Final avg loss   0.389069483943, Improvement -0.00623041269697
Hidden Size  2, alpha  0.1: Final avg loss   0.360967294585, Improvement -0.0274751908786
Hidden Size  2, alpha    1: Final avg loss   0.225124983188, Improvement -0.134245541987
Hidden Size  2, alpha   10: Final avg loss     0.0212916047, Improvement -0.183871680009
Hidden S

#### Aftermath

TODO alpha, layer size

### Deep Q Network Step by Step. Step 1: Deep Q Learning from Single Observations

TODO what changes

#### Namespace (Extension)

In [31]:
from keras.layers import Dense
from keras.models import Sequential

#### Set Hyperparameters (Extension)

We start with a first set of static hyperparameters. Some of them will undergo changes along the way:

- *D_RANGE* is the number of frames the agent should take into account as the current state it is in: This is the "operational" memory of the agent. I will refer to it as frame memory.
- *HIDDEN_DIM* is the number of neurons in the hidden layer
- Note that we do not use *ALPHA*, *Q_TABLE*, and *Q_INIT* anymore

In [33]:
D_RANGE = np.arange(1, 31) # Constant over one epoch
HIDDEN_DIM = 200

#### Build Keras Model

TODO
- one fully connected layer
- multicat output (softmax)

In [34]:
def _create_network():
    model = Sequential()
    model.add(Dense(HIDDEN_DIM, init='glorot_normal', input_shape=(D*INPUT_DIM,))) #default is: init='glorot_uniform' 
    model.add(Dense(N_ACTIONS, init='glorot_normal', activation='softmax')) # default is: init='glorot_uniform' 
    model.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    return model

#### Build Training Epoch

In [35]:
def train_dqn1(model, render):
    # Start a new epoch
    episode = 0; success = 0; solved = False

    for episode in range(N_EPISODES):
        x_t = ENV.reset()
        # Pile up four times the same init observation, in order to be consistent with the model input
        s_t = np.tile(x_t, D)
        done = False

        # Start an episode
        while not done:
            # Reset the environment for a new gaming step
            if render: ENV.render()
    
            # Estimate rewards for each action (targets), at s_t
            q = model.predict(s_t[np.newaxis])[0] 

            # Take action with highest estimated reward (argmax returns index)
            a_t = np.argmax(q) 
            
            # Observe after action a_t
            x_t, r_t, done, info = ENV.step(a_t) 
            
            # Create state at t1: Append x observations, throw away the earliest
            s_t1 = np.concatenate((x_t, s_t[:(D-1) * INPUT_DIM,]), axis=0) 

            # Estimate rewards for each action (targets), at s_t1 (again a forward pass)
            Q_sa = model.predict(s_t1[np.newaxis])[0] 

            ''' Create reference/targets by updating estimated reward for chosen action:
                For action taken, replace estimated reward by remaining cumulative lifetime reward
            ''' 
            targets = q
            targets[a_t] = r_t + GAMMA * np.max(Q_sa) if not done else r_t

            ''' Learn!
                - Again, predict q values for state s_t
                - Calculate loss by comparing predictions to targets: they will differ only for the action taken
                - backpropagate error for action taken, update weights
            ''' 
            model.fit(s_t[np.newaxis], targets[np.newaxis], nb_epoch=10, verbose=0)

            # Update state
            s_t = s_t1
        
            # Bookkeeping
            if r_t >= 100: success += 1
            if r_t >= 200: solved = True

    print "D {}, Successes {}, Solved {}".format(D, success, solved)

#### Train Model

What can learn from this basic implementation? Surely, does it make the impression to learn at all? We also should think of the agent's frame memory. Does it make sense to let the agent know only its current state, or shall we allow him to take into consideration also some of the states before? If yes, how far back should it remember? To get a first hint to answer this question, let's run a simulation. We loop over a range of D candidates, and produce some summary statistics in order to assess the performance: 

In [13]:
'''
for D in D_RANGE:
    model = _create_network() # Initialize model
    train_dqn1(model, render=False) # Train model
'''

'\nfor D in D_RANGE:\n    model = _create_network() # Initialize model\n    train_dqn1(model, render=False) # Train model\n'

#### Aftermath

That is quite working nicely already! The frantic, purpose-less behaviour is gone most of the time, or is vanishing quickly within a few episodes only.

Some observations: 
- Too little frame memory? With only one frame in the frame memory, I often observed extensive swinging movement: the agent tries to correct with the left or right engine, fires too much, the lander is tipping over to the opposite side. Then, it corrects again, and again, and from all these corrections forgets to fire against the moon's gravity, and the agent is crashing into the surface. TODO Which are the best?
- Local minima. There are plenty of times one can see the agent trapped into a locally optimal policy. For example, it stays on the ground, engaging left and right engine forever, perfectly stable, but not reaching the ultimate goal. Or a setting where left and right engines are engaged, but the lower engine does not fire at all, over long episodes.

### Deep Q Network Step by Step.  Step 2: *Epsilon Greedy* Deep Q Learning from Single Observations

At this point, let us tackle the issue of getting stuck in local minima. As a remedy, Reinforcement Learning makes use the so called *epsilon greedy* action selection policy. It allows for a random move with probability *epsilon*, and by that introduces the notion of exploration (random moves) vs. exploitation (act on estimation of the q function). 

*Exploration* will reduce the probability of getting stuck in local minima, which are *not* reflecting the best action given a certain experience level of the agen. It's just like fresh air for the AI brain, introducing random ideas from outside. 

On the other hand, the agent needs to train and get experience with his selected moves. It needs evidence that one decision was (not) the right one, and to update its decicion finding process (the weights of the ANN. It only gets it by acting according to its own decisions undisturbed by random inputs. This is where *exploitation* comes in.

In RL, usually *epsilon* decreases over a certain exploration period. This reflects the idea that the agent will start with many random moves to fathom the environment by just observing. With time and growing experience, it will decrease the share of random moves, since it feels more confident in its own decisions. 

I will follow th custom of allowing random moves at a linearly decreasing exploration rate during the exploration period. Deep Mind: Playing Atari with Deep Reinforcement Learning (p.6) is starting with *epsilon* = 1 / complete randomness. Let us see what brings good results here. Balancing exploration and exploitation is in itself subject to learning.

Concretely, we now define a interval between the maximum and minimum *epsilon* allowed: *EPSILON_RANGE*.

In order to implement exploration and exploitation, we need to keep track of the number of frames only. We thus stop counting episodes (*N_EPISODE*) and establish global settings for the total number of frames (*TOTAL_FRAMES*) and the total number of exploration frames (*TOTAL_EXPLORATION_STEPS*). 

Following Deep Mind: Playing Atari with Deep Reinforcement Learning (p.6), *TOTAL_FRAMES* is set to 1e7. 1/10th of *TOTAL_FRAMES* is assigned to exploration, the rest to training.

During training, the agent will run on the minimum *epsilon* constantly.

#### Set Hyperparameters (Extension)

In [14]:
TOTAL_FRAMES = 1e7
TOTAL_EXPLORATION_STEPS = TOTAL_FRAMES / 10
EPSILON_RANGE = [1, 0.01]

#### Build Training Epoch

In [15]:
def train_dqn2(model, render): 
    # Start a new epoch
    t = 0; success = 0; solved = False
    epsilon = EPSILON_RANGE[0] ##NEW Initialize epsilon at its maximum value

    #for episode in range(N_EPISODES): ###NEW Discard the episode loop
    while t <= TOTAL_FRAMES: ###NEW Install a loop over all frames
        x_t = ENV.reset()
        s_t = np.tile(x_t, D)
        done = False

        # Start an episode
        while not done:
            t += 1
            
            # Reset the environment for a new gaming step
            if render: ENV.render()
            
            ###NEW Anneal random exploration rate epsilon over exploration period
            epsilon = epsilon - epsilon / TOTAL_EXPLORATION_STEPS if epsilon > EPSILON_RANGE[1] else EPSILON_RANGE[1]
    
            # Estimate rewards for each action (targets), at s_t
            q = model.predict(s_t[np.newaxis])[0]

            # Take action with highest estimated reward 
            ###NEW Do this with probability 1-epsilon ("epsilon greedy" policy)
            a_t = np.argmax(q) if np.random.random() > epsilon else np.random.choice(ACTIONS, 1)[0]

            # Observe after action a_t
            x_t, r_t, done, info = ENV.step(a_t)
        
            # Create state at t1: Append x observations, throw away the earliest
            s_t1 = np.concatenate((x_t, s_t[:(D-1) * INPUT_DIM,]), axis=0)

            # Estimate rewards for each action (targets), at s_t1 (again a forward pass)
            Q_sa = model.predict(s_t1[np.newaxis])[0]

            ''' Create reference/targets by updating estimated reward for chosen action
                For action taken, replace estimated reward by remaining cumulative lifetime reward
            ''' 
            targets = q
            targets[a_t] = r_t + GAMMA * np.max(Q_sa) if not done else r_t

            ''' Learn!
                - Again, predict q values for state s_t
                - Calculate loss by comparing predictions to targets: they will differ only for the action taken
                - backpropagate error for action taken, update weights
            ''' 
            model.fit(s_t[np.newaxis], targets[np.newaxis], nb_epoch=10, verbose=0)

            # Update state
            s_t = s_t1
        
            # Bookkeeping
            if r_t >= 100: success += 1
            if r_t >= 200: solved = True

    print "D {}, Successes {}, Solved {}".format(D, success, solved)

#### Train Model

In [18]:
'''
for D in D_RANGE:
    # Initialize model
    model = _create_network()
    # Train model
    train_dqn2(model, render=True)
'''

'\nfor D in D_RANGE:\n    # Initialize model\n    model = _create_network()\n    # Train model\n    train_dqn2(model, render=True)\n'

### Deep Q Network Step by Step. Step 3: Deep Q Learning *from Stored Experiences* 

Deep Mind: Playing Atari with Deep Reinforcement Learning claims that "learning directly from consecutive samples is inefficient, due to the strong correlations between the samples" (p.5). Instead, the trick is to learn from a memory storage, which I will call Experience Replay Memory (*ERM*), in batches. 

We are thus going to create the main database of the agent: It is the place where it 
- stores states and its experiences with the states (transitions *s_t*, *a_t*, *r_t*,and *s_t1*)
- recalls on the memory, collects a memory sample, trains on the sample, and updates the Q function estimator.

The *ERM* is set up once per epoch and is fed at each frame with fresh transition evidence.

#### Namespace (Extension)

In [19]:
from collections import deque

#### Set Hyperparameters (Extension)

*ERM_SIZE* is setting the size of the experience replay memory. Following the recommendation of Deep Mind: Playing Atari with Deep Reinforcement Learning (p.6), we set it equal to the number of exploration steps. *BATCH_SIZE* denotes the size of the transition sample which is drawn uniformly without replacement from the *ERM* at each frame. Again, its size is following the recommendations of Deep Mind: Playing Atari with Deep Reinforcement Learning (p.6).

In [20]:
ERM_SIZE = TOTAL_EXPLORATION_STEPS
BATCH_SIZE = 32

#### Build Training Epoch

In [21]:
def train_dqn3(model, render):
    # Start a new epoch
    t = 0; success = 0; solved = False
    epsilon = EPSILON_RANGE[0]
    ERM = deque(maxlen=ERM_SIZE) ###NEW If too long, throw away the earliest (latest is ERM[-1])

    # Do not count episodes per epoch, but frames
    while t <= TOTAL_FRAMES:
        x_t = ENV.reset()
        s_t = np.tile(x_t, D)
        done = False

        # Start an episode
        while not done:
            t += 1
            
            # Reset the environment for a new gaming step
            if render: ENV.render()
            
            # Anneal random exploration rate epsilon over exploration period
            epsilon = epsilon - epsilon / TOTAL_EXPLORATION_STEPS if epsilon > EPSILON_RANGE[1] else EPSILON_RANGE[1]
    
            # Estimate rewards for each action (targets), at s_t
            q = model.predict(s_t[np.newaxis])[0]

            # Take action with highest estimated reward, "epsilon greedy"
            a_t = np.argmax(q) if np.random.random() > epsilon else np.random.choice(ACTIONS, 1)[0]

            # Observe after action a_t
            x_t, r_t, done, info = ENV.step(a_t)
        
            # Create state at t1: Append x observations, throw away the earliest
            s_t1 = np.concatenate((x_t, s_t[:(D-1) * INPUT_DIM,]), axis=0)
            
            ###NEW Store transition in experience replay memory
            ERM.append((s_t, a_t, r_t, s_t1))

            ###NEW Choose a batch of maximum length BATCH_SIZE
            minibatch = np.array([ ERM[i] for i in np.random.choice(np.arange(0, len(ERM)), min(len(ERM), BATCH_SIZE)) ])
            
            ###NEW Compute targets/reference for each transition in minibatch
            inputs = deque(); targets = deque()
            for m in minibatch:
                inputs.append(m[0]) # Append s_t of batch transition m to inputs
                m_q    = model.predict(m[0][np.newaxis])[0] # Estimate rewards for each action (targets), at s_t
                m_Q_sa = model.predict(m[3][np.newaxis])[0] # Estimate rewards for each action (targets), at s_t1
                m_targets = m_q
                m_targets[m[1]] = m[2] + GAMMA * np.max(m_Q_sa)
                targets.append(m_targets) # Append target of batch transition m to targets
                
            ###NEW Train the model by backpropagating the errors and update weights
            model.train_on_batch(np.array(inputs), np.array(targets))
            
            # Update state
            s_t = s_t1
        
            # Bookkeeping
            if r_t >= 100: success += 1
            if r_t >= 200: solved = True

    print "D {}, Successes {}, Solved {}".format(D, success, solved)

#### Train Model

In [ ]:
'''
for D in D_RANGE:
    # Initialize model
    model = _create_network()
    # Train model
    train_dqn3(model, render=True)
'''

#### Aftermath

The question crossed my mind: Why don't we predict beforehand, on the fly, at every frame? Would that not be computationally efficient? This has the huge disadvantage that we predict with the knowledge available at timestep t. This might be faulty, and the faulty prediction stays as target reference in the batch, and is used to compare the loss for the taken action between the prediction at timestep t and the potenitially long ago target estimation. This will bias the learning process significantly. Thus, we select a batch, calculate the estimations and targets for the complete batch, both with the knowledge of the current frames.

### Deep Q Network Step by Step. Step 4: Deep Q Learning from Stored Experiences, *Refined*

At this point, we will finalize training by applying some more tweaks, outside and within the model block. The goal is to check parameter combinations by brute force. We install some extra bookeeping, and will save a model for every training method.

Most importantly, we establish two new parameters:

- Frames per Action (*FPA*) determines how many frames pass before a fresh action is taken. Deep Mind - Playing Atari with Deep Reinforcement Learning suggests 4 skipping 4 time frames before conducting an action (p.6).
- Reward Clipping (*R_CLIP*) is also an idea taken from Deep Mind - Playing Atari with Deep Reinforcement Learning (p.6), where it was implemented in order to make rewards comparable across the different games.

Besides that, we gather statistics:

- The average reward per episode (*episode_avg_rewards*)
- The number of frames per episode (*episode_lengths*)

Finally, we establish a parameter testing routine, which will go through parameter settings with brute force. For each of the parameter settings, the weights and the model are saved. Once a well-working setting is obtained, we will have the opportunity of directly running from the pre-learned weights.

#### Namespace (Extension)

In [26]:
from sys import stdout
import json
from keras.models import model_from_json
from os import path, makedirs

#### Set Hyperparameters (Extension)

- Discount factor gamma. We replace the fixed hyperparameter by an arbitrary range of candidates (*GAMMA_RANGE*).
- Take action every n-th frame (frame per action *FPA_RANGE*)
- Reward Clipping (*R_CLIP*)
- Number of neurons in the hidden layer. We replace the fixed parameter by an arbitrary range of candidates (*HIDDEN_DIMS*)
- *SAVE_PATH* is the working directory where the training epoch models are stored

In [38]:
GAMMA_RANGE = np.arange(0.0, 1.01, 0.01) # Constant over one epoch
FPA_RANGE = np.arange(1, 5) # Constant over one epoch
R_CLIP = [False, True] # Constant over one epoch
HIDDEN_DIMS = np.arange(10, 400, 10) 
SAVE_PATH = path.join(path.expanduser("~"), "Models", "DQN")
try: 
    makedirs(SAVE_PATH)
except OSError:
    if not path.isdir(SAVE_PATH):
        raise

#### Build Training Epoch

In [39]:
def train_dqn4(model, render):
    # Start a new epoch
    t = 0; success = 0; solved = False
    epsilon = EPSILON_RANGE[0]
    ERM = deque(maxlen=ERM_SIZE)
    MH5 = path.join(SAVE_PATH, MODEL_ID+".h5") ###NEW Define path and name of h5 container
    MJS = path.join(SAVE_PATH, MODEL_ID+".json") ###NEW Define path and name of json container
    episode_avg_rewards = list() ###NEW Statistics
    episode_t = deque([0, 0], maxlen=2) ###NEW Statistics
    episode_lengths = list() ###NEW Statistics

    # Do not count episodes per epoch, but frames
    while t <= TOTAL_FRAMES:
        x_t = ENV.reset()
        s_t = np.tile(x_t, D)
        done = False
        episode_rewards = [] ###NEW Statistics

        # Start an episode
        while not done:
            t += 1
            
            # Reset the environment for a new gaming step
            if render: ENV.render()
            
            # Anneal random exploration rate epsilon over exploration period
            epsilon = epsilon - epsilon / TOTAL_EXPLORATION_STEPS if epsilon > EPSILON_RANGE[1] else EPSILON_RANGE[1]
    
            # Estimate rewards for each action (targets), at s_t
            q = model.predict(s_t[np.newaxis])[0]

            # Take action with highest estimated reward, "epsilon greedy"
            ###NEW Act only every n-th frame
            if t % FPA == 0: a_t = np.argmax(q) if np.random.random() > epsilon else np.random.choice(ACTIONS, 1)[0]

            # Observe after action a_t
            x_t, r_t, done, info = ENV.step(a_t)
            
            ###NEW Clip rewards
            if R_CLIP and r_t != 0: r_t = abs(r_t) / r_t
            
            # Create state at t1: Append x observations, throw away the earliest
            s_t1 = np.concatenate((x_t, s_t[:(D-1) * INPUT_DIM,]), axis=0)
            
            # Store transition in experience replay memory
            ERM.append((s_t, a_t, r_t, s_t1))

            # Choose a batch of maximum length BATCH_SIZE
            minibatch = np.array([ ERM[i] for i in np.random.choice(np.arange(0, len(ERM)), min(len(ERM), BATCH_SIZE)) ])
            
            # Compute targets/reference for each transition in minibatch
            inputs = deque(); targets = deque()
            for m in minibatch:
                inputs.append(m[0]) # Append s_t of batch transition m to inputs
                m_q    = model.predict(m[0][np.newaxis])[0] # Estimate rewards for each action (targets), at s_t
                m_Q_sa = model.predict(m[3][np.newaxis])[0] # Estimate rewards for each action (targets), at s_t1
                m_targets = m_q
                m_targets[m[1]] = m[2] + GAMMA * np.max(m_Q_sa)
                targets.append(m_targets) # Append target of batch transition m to targets
                
            # Train the model by backpropagating the errors and update weights
            model.train_on_batch(np.array(inputs), np.array(targets))
                
            ###NEW Save progress every 100 iterations
            if t % 100 == 0:
                model.save_weights(MH5, overwrite=True)
            with open(MJS, "w") as outfile: json.dump(model.to_json(), outfile)
            
            # Update state
            s_t = s_t1
        
            # Bookkeeping
            if r_t >= 100: success += 1
            if r_t >= 200: solved = True
            episode_rewards.append(r_t) ###NEW Statistics

        # Epoch Bookkeeping 
        episode_avg_rewards.append(np.mean(np.array(episode_rewards))) ###NEW Statistics
        episode_t.append(t) ###NEW Statistics
        episode_lengths.append(episode_t[1] - episode_t[0]) ###NEW Statistics
                  
    # Return values for epoch's bookkeeping
    return success, solved, episode_avg_rewards, episode_lengths

#### Train Model

In [ ]:
max_success = 0 
best_epochs = deque(maxlen=50) # Get the n best performing parameter settings

print "Session contains {} epochs\n".format(len(D_RANGE)*len(GAMMA_RANGE)*len(FPA_RANGE)*len(R_CLIP)*len(HIDDEN_DIMS))

for D in D_RANGE:
    # Initialize model
    model = _create_network()
    for GAMMA in GAMMA_RANGE: 
        for FPA in FPA_RANGE:
            for R_C in R_CLIP:
                for HIDDEN_DIM in HIDDEN_DIMS:
                    # Create model id
                    MODEL_ID = repr(D)+"_"+str(round(GAMMA, 2))+"_"+repr(FPA)+"_"+repr(R_C)[:1]
                    # Train model
                    success, solved, episode_avg_rewards, episode_lengths = train_dqn4(model, render=True) 
                    stdout.write("\rEpoch: {}, D: {}, Gamma: {}, Frames per action: {}, Reward clipping: {}, Neurons in hidden layer: {}, Successes: {}, Solved: {}".format( \
                                                            epoch, \
                                                            D, \
                                                            round(GAMMA, 2), \
                                                            FPA, \
                                                            R_C, \
                                                            HIDDEN_DIM, \
                                                            success, \
                                                            solved))
                    stdout.flush()
                    if solved or (success >= max_success and success > 0):
                        max_success = success
                        best_epochs.append((D, GAMMA, FPA, R_C, HIDDEN_DIM, success, solved, episode_avg_rewards, episode_lengths))
                        print "\n\nBest Epochs:"
                        for be in best_epochs: print be
                        print "\n"
                        if solved: break  

Session contains 945360 epochs



In [44]:
'''
    if args['mode'] == 'Run':
        OBSERVE = 999999999    #We keep observe, never train
        epsilon = FINAL_EPSILON
        print ("Now we load weight")
        model.load_weights("model.h5")
        adam = Adam(lr=1e-6)
        model.compile(loss='mse',optimizer=adam)
        print ("Weight load successfully")  
'''

## erst jetzt model save, nachdem die hyperparameter-exploration vorbei ist?

'\n    if args[\'mode\'] == \'Run\':\n        OBSERVE = 999999999    #We keep observe, never train\n        epsilon = FINAL_EPSILON\n        print ("Now we load weight")\n        model.load_weights("model.h5")\n        adam = Adam(lr=1e-6)\n        model.compile(loss=\'mse\',optimizer=adam)\n        print ("Weight load successfully")  \n'